# Instalation for Conda and PIP3

In [3]:
!conda install -c anaconda pandas -y
!conda install -c anaconda fse -y
!conda install -c conda-forge matplotlib -y
!pip3 install fse
!pip3 install pytorch-nlp
!pip3 install nltk

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.14.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/nurrizkyimani/.conda/envs/workspace

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           5 KB  anaconda
    bottleneck-1.3.4           |   py38h67323c0_0         129 KB  anaconda
    ca-certificates-2022.4.26  |       hecd8cb5_0         132 KB  anaconda
    certifi-2022.6.15          |   py38hecd8cb5_0         157 KB  anaconda
    intel-openmp-2021.4.0      |    hecd8cb5_3538         1.2 MB  anaconda
    mkl-2021.4.0               |     hecd8cb5_637       209.2 MB  anaconda
    mkl-service-2.4.0          |   py38h9ed2024_0          48 KB  an

# Import : Dependency

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import re
import string
import collections
import fse
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchnlp.metrics import get_moses_multi_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

import nltk
nltk.download('punkt')

# Any results you write to the current directory are saved as output.

/Users/nurrizkyimani/.conda/envs/workspace/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nurrizkyimani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Path: Local Path for taking the data

In [2]:
all_agree_path = "2022_thesis_styletransfer/annotated/combined/csv/all_agree.csv"
all_agree_pd = pd.read_csv(all_agree_path)
df_clickbait = all_agree_pd[all_agree_pd['label'] == "clickbait"]
df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
#creating a concatination of both df cickbait and nonclickbait
df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)
df_both.head(-2)

df_both_l = df_both.copy()
df_both_l['label'] = df_both_l['label'].astype('category')
df_both_l.head(-1)

,title,label,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,1
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,1
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",clickbait,1
...,...,...,...
8607,"Smart SIM Diluncurkan, Wakapolri Harap Bisa Me...",non-clickbait,0
8608,"Wamena Papua Kembali Membara, Kantor Bupati Di...",non-clickbait,0
8609,BMKG Angkat Bicara Soal Langit Merah Jambi,non-clickbait,0
8610,Operasional Bandara Wamena Dihentikan Akibat R...,non-clickbait,0


In [3]:
df_both_label = df_both_l.drop('label_score', axis=1)
df_both = df_both_l.drop('label', axis=1)
df_both.head(-5)

,title,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,1
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,1
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",1
...,...,...
8603,Komnas HAM Minta Aparat Keamanan Tak Gunakan K...,0
8604,Mahasiswa Minta TNI Turun Bersama Kawal Aksi D...,0
8605,Polisi Pulangkan 56 Mahasiswa Setelah Sempat D...,0
8606,KPK Tepis Tudingan Moeldoko Soal Hambat Investasi,0


In [4]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df_both_l['label_encoding'] = labelencoder.fit_transform(df_both_l['label'])

df_encode_clean = df_both_l.copy()
df_encode_clean = df_both = df_both_l.drop(['label_score'], axis=1)
df_encode_clean.head()

,title,label,label_encoding
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,0
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,0
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,0
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,0
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",clickbait,0


# Preprocessing: lower casing

In [5]:
# preprocessing : lower casing
df_encode_clean['title'] = df_encode_clean['title'].str.lower()
df_encode_clean.title

0       viral! driver ojol di bekasi antar pesanan mak...
1       ada motor nyangkut di atas bambu di sleman, ko...
2            pesan gamblang poyuono menolak revisi uu kpk
3       kocak! maling di rumah mewah jakut terekam cct...
4       viral video diduga baku tembak di sleman, ini ...
                              ...                        
8608    wamena papua kembali membara, kantor bupati di...
8609           bmkg angkat bicara soal langit merah jambi
8610    operasional bandara wamena dihentikan akibat r...
8611     asap karhutla riau mulai merambah ke nias, bm...
8612    tolak ruu pertanahan, ribuan petani siap gelar...
Name: title, Length: 8613, dtype: object

In [6]:
df_encode_clean_t =  df_encode_clean.copy()
df_encode_clean_t['title'] = df_encode_clean_t['title'].apply(lambda y: " ".join((re.sub(r'([!/?/./,/%/\'/\"/-/:])', lambda x: ' ' + x.group()+' ' , y)).split()) , 1)
df_encode_clean_t

,title,label,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,clickbait,0
1,"ada motor nyangkut di atas bambu di sleman , k...",clickbait,0
2,pesan gamblang poyuono menolak revisi uu kpk,clickbait,0
3,kocak ! maling di rumah mewah jakut terekam cc...,clickbait,0
4,"viral video diduga baku tembak di sleman , ini...",clickbait,0
...,...,...,...
8608,"wamena papua kembali membara , kantor bupati d...",non-clickbait,1
8609,bmkg angkat bicara soal langit merah jambi,non-clickbait,1
8610,operasional bandara wamena dihentikan akibat r...,non-clickbait,1
8611,"asap karhutla riau mulai merambah ke nias , bm...",non-clickbait,1


create a space between the punctuation, DO NOT RUN TWICE

preprocessing: split between clickbait and nonclickbait

In [7]:
df_clickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "clickbait"]
df_nonclickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "non-clickbait"]

make the df into a list only in clickbait and nonclickbait

In [20]:
d_both = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both = d_both.drop(['label', 'label_encoding'], axis=1).values.tolist()
d_both = list(itertools.chain.from_iterable(d_both))

In [21]:
d_both

['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda',
 'ada motor nyangkut di atas bambu di sleman , kok bisa ?',
 'pesan gamblang poyuono menolak revisi uu kpk',
 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
 'viral video diduga baku tembak di sleman , ini kata polisi',
 'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta',
 'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini rinciannya',
 'sering quality time bersama keluarga ? ternyata ini 3 manfaatnya !',
 'menteri jokowi 55 % profesional , ini kandidatnya ?',
 "pria misterius berjubah putih viral di sumut , ingatkan ' tuhan murka '",
 'video porno di sumedang disebar pemeran pria , apa motifnya ?',
 'terungkap ! video porno di sumedang diperankan pasangan selingkuh',
 'kendaraan listrik bebas dari aturan ganjil-genap , ini alasannya',
 'menanti " kebijakan gila " jokowi',
 'akhirnya ! pemkot pekanbaru liburkan sd dan smp karena kabut asap',
 '2 ribu hekt

In [8]:
# make the df into a list only in clickbait and nonclickbait

import itertools

df_clickbait_t = df_clickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
clickbait_l_c = list(itertools.chain.from_iterable(df_clickbait_t))

df_nonclickbait_t = df_nonclickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
nonclickbait_l_c = list(itertools.chain.from_iterable(df_nonclickbait_t))

print(nonclickbait_l_c[0:5])
print(clickbait_l_c[0:5])

['masuk radar pilwalkot medan , menantu jokowi bertemu dpw nasdem sumut', 'malaysia sudutkan ri : isu kabut asap hingga invasi babi', 'kemensos salurkan rp 7 , 3 m bagi korban kerusuhan sosial di papua', 'mpr : amandemen uud 1945 tak akan melebar ke mana-mana', 'peringati tahun baru islam , banyuwangi kembali gelar festival muharam']
['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda', 'ada motor nyangkut di atas bambu di sleman , kok bisa ?', 'pesan gamblang poyuono menolak revisi uu kpk', 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur', 'viral video diduga baku tembak di sleman , ini kata polisi']


Creating the iterators and split training, test, and validation

In [9]:
# end result : list
nonclickbait_l_c
clickbait_l_c

['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda',
 'ada motor nyangkut di atas bambu di sleman , kok bisa ?',
 'pesan gamblang poyuono menolak revisi uu kpk',
 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
 'viral video diduga baku tembak di sleman , ini kata polisi',
 'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta',
 'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini rinciannya',
 'sering quality time bersama keluarga ? ternyata ini 3 manfaatnya !',
 'menteri jokowi 55 % profesional , ini kandidatnya ?',
 "pria misterius berjubah putih viral di sumut , ingatkan ' tuhan murka '",
 'video porno di sumedang disebar pemeran pria , apa motifnya ?',
 'terungkap ! video porno di sumedang diperankan pasangan selingkuh',
 'kendaraan listrik bebas dari aturan ganjil-genap , ini alasannya',
 'menanti " kebijakan gila " jokowi',
 'akhirnya ! pemkot pekanbaru liburkan sd dan smp karena kabut asap',
 '2 ribu hekt

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

# Ngram : Delete, get attribute and content section

In [10]:
#Parameters:
param_smooth = 1
param_threshold = 5
param_span = 4

param_backoff_limit = 3

#ngram has punctuation
def has_punctuation(ngram): #damn I'm very proud of making this from scratch lol, looks elegant in one line
    return True in [x in string.punctuation for x in ngram]

# generate ngram from all of the sentence
def generate_ngrams(lines,  min_length: int =1, max_length=param_span) -> dict :
    #     lines = placeholder + lines
    lengths: range  = range(min_length, max_length + 1)
    ngrams: dict  = {length: [] for length in lengths}
    queue: collections.deque = collections.deque(maxlen=max_length)

    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length and not has_punctuation(current[:length]):
                ngrams[length].append(current[:length])

    short_by = 0
    for line in lines:
        short_by = max(0, max_length - len(lines))
        for word in line.split():
            queue.append(word)
            if len(queue) >= max_length-short_by:
                add_queue()

    while len(queue) > min_length:
        queue.popleft()
        add_queue()
    return ngrams

#modified from & fixed their error of ngram with # of words < 4: https://gist.github.com/benhoyt/dfafeab26d7c02a52ed17b6229f0cb52
def count_ngrams(lines, min_length: int =1, max_length=param_span ) -> dict:
    """Iterate through given lines iterator (file object or list of
    lines) and return n-gram frequencies. The return value is a dict
    mapping the length of the n-gram to a collections.Counter
    object of n-gram tuple and number of times that n-gram occurred.
    Returned dict includes n-grams of length min_length to max_length.
    """
    lengths = range(min_length, max_length + 1)
    ngrams = {length: collections.Counter() for length in lengths}
    queue = collections.deque(maxlen=max_length)

    # Helper function to add n-grams at start of current queue to dict
    def add_queue():
        current = tuple(queue)
        for length in lengths:
            if len(current) >= length and not has_punctuation(current[:length]):
                ngrams[length][current[:length]] += 1

    # Loop through all lines and words and add n-grams to dict
    short_by = 0
    for line in lines:
        short_by = max(0, max_length - len(lines))
        for word in line.split():
            queue.append(word)
            if len(queue) >= max_length - short_by:
                add_queue()

    # Make sure we get the n-grams at the tail end of the queue
    while len(queue) > min_length:
        queue.popleft()
        add_queue()

    return ngrams


In [11]:
# EXAMPLE : generate ngram without any counting; just generate ngram
test = generate_ngrams(clickbait_l_c)
test

{1: [('viral',),
  ('driver',),
  ('ojol',),
  ('di',),
  ('bekasi',),
  ('antar',),
  ('pesanan',),
  ('makanan',),
  ('pakai',),
  ('sepeda',),
  ('ada',),
  ('motor',),
  ('nyangkut',),
  ('di',),
  ('atas',),
  ('bambu',),
  ('di',),
  ('sleman',),
  ('kok',),
  ('bisa',),
  ('pesan',),
  ('gamblang',),
  ('poyuono',),
  ('menolak',),
  ('revisi',),
  ('uu',),
  ('kpk',),
  ('kocak',),
  ('maling',),
  ('di',),
  ('rumah',),
  ('mewah',),
  ('jakut',),
  ('terekam',),
  ('cctv',),
  ('bingung',),
  ('cari',),
  ('jalan',),
  ('kabur',),
  ('viral',),
  ('video',),
  ('diduga',),
  ('baku',),
  ('tembak',),
  ('di',),
  ('sleman',),
  ('ini',),
  ('kata',),
  ('polisi',),
  ('waspada',),
  ('ada',),
  ('penipuan',),
  ('catut',),
  ('pertamina',),
  ('korbannya',),
  ('rugi',),
  ('puluhan',),
  ('juta',),
  ('jaksa',),
  ('beberkan',),
  ('senpi-peluru',),
  ('tajam',),
  ('pembelian',),
  ('kivlan',),
  ('zen',),
  ('cs',),
  ('ini',),
  ('rinciannya',),
  ('sering',),
  ('quality

In [12]:
#Example of usage: count ngram of the list of sentence
d_nonclick_ngrams_counts = count_ngrams(nonclickbait_l_c)
d_clickbait_ngrams_counts = count_ngrams(clickbait_l_c)
d_nonclick_ngrams_counts

{1: Counter({('masuk',): 35,
          ('radar',): 1,
          ('pilwalkot',): 5,
          ('medan',): 19,
          ('menantu',): 2,
          ('jokowi',): 174,
          ('bertemu',): 22,
          ('dpw',): 1,
          ('nasdem',): 5,
          ('sumut',): 20,
          ('malaysia',): 44,
          ('sudutkan',): 1,
          ('ri',): 39,
          ('isu',): 6,
          ('kabut',): 113,
          ('asap',): 163,
          ('hingga',): 77,
          ('invasi',): 1,
          ('babi',): 2,
          ('kemensos',): 2,
          ('salurkan',): 16,
          ('rp',): 94,
          ('7',): 32,
          ('3',): 88,
          ('m',): 42,
          ('bagi',): 27,
          ('korban',): 61,
          ('kerusuhan',): 16,
          ('sosial',): 7,
          ('di',): 1348,
          ('papua',): 130,
          ('mpr',): 15,
          ('amandemen',): 1,
          ('uud',): 3,
          ('1945',): 1,
          ('tak',): 218,
          ('akan',): 174,
          ('melebar',): 1,
          ('ke',

In [13]:
#these are methods that will become useful when extracting attribute markers
#why do we need all this? well... that's like 5 hours of debugging...
def flatten(foo):
    return list(_flatten(foo))

def _flatten(foo):
    for x in foo:
        if isinstance(x, collections.Iterable) and not isinstance(x, str):
            for y in _flatten(x):
                yield y
        else:
            yield x

def array_to_string(a):
    return ' '.join(flatten(a))

def is_in_string_array(elements, original): #deprecated, does not take into account sequence order
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()

# no usage in the func
def insert_string(string, inserted_string, index):
    return string[:index] + inserted_string + string[index:]

# modified from https://stackoverflow.com/questions/41752946/replacing-a-character-from-a-certain-index
def replace_string(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string. index:" + index)

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + len(newstring):]

Get attribute marker, content

In [14]:
# get the ngram count based on the reference sentence onlhy
def get_counts(list1, counted_ngrams):
    counts = []
    list1_ngrams = generate_ngrams(list1)
    list2_counts = counted_ngrams

    for length in range(param_span,0, -1):
        for v in list1_ngrams[length]:
            counts.append([list2_counts[length][v], v])
    return np.array(counts)

def get_attribute_markers(s, style_src):
    sentence = [s]

    ngrams = get_counts(sentence, d_nonclick_ngrams_counts)
    if len(ngrams) > 0:
        ngrams = ngrams[:,1]

    pos_counts = get_counts(sentence, d_nonclick_ngrams_counts)
    if len(pos_counts) > 0:
        pos_counts = pos_counts[:,0]

    neg_counts = get_counts(sentence, d_clickbait_ngrams_counts)
    if len(neg_counts) > 0:
        neg_counts = neg_counts[:,0]


    if(style_src):
        importances = (pos_counts + param_smooth) / (neg_counts + param_smooth)
    else:
        importances = (neg_counts + param_smooth) / (pos_counts + param_smooth)

    a = []

    importances = np.vstack((importances, ngrams)).T
    for importance in importances:
        if importance[0] > param_threshold and not is_in_string_array(importance[1], a):
            a.append(' '.join(importance[1]))
    return a

def separate(sentence, style_src):
    attributes = get_attribute_markers(sentence, style_src)
    c = sentence

    replace_indexes = []
    for a in attributes:
        replace_index = -1
        replace_index = c.find(a)
        replace_indexes.append(replace_index)
        c = c.replace(a, " "*len(a))

    if len(attributes) == 0:
        return {'c': c, 'a': [], 'i': [], 's': sentence}

    replace_indexes, attributes = zip(*sorted(zip(replace_indexes, attributes)))
    return {'c': c, 'a': attributes, 'i': replace_indexes, 's': sentence}

def get_c(sentence, style):
    return re.sub(' +', ' ', separate(sentence, style)['c'])

def get_a(sentence, style):
    a = separate(sentence, style)['a']
    if len(a) > 0:
        return ' '.join(a)
    else:
        return ""

# TFIDF : Get similar sentence

In [15]:
# ===== TF-IDF Weighted Word Overlap ===== #
# docs pre-processing
docs = d_both.tolist()

# creating dict_idf = {word: idf}
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(use_idf=True, stop_words=None)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
dict_idf = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

def get_overlap(a, b):
    #     print(a, b)
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_weighted_overlap(a, b):
    overlap = get_overlap(a, b)
    a_counter = collections.Counter(a.split())
    #calculate
    weighted_overlap = 0
    for word in overlap:

        word_tf = a_counter[word]#/len(a.split()) -> commented out cause division by constant value doesn't matter

        get_idf = dict_idf.get(word)
        word_idf = 1 if get_idf == None else get_idf #get rid of error when idf not in dict

        word_tfidf = word_tf*word_idf
        weighted_overlap+=overlap[word]*word_tfidf

    return weighted_overlap

def get_closest_sentence_tfidf(sentence, style_src):
    opposite_dataset = d_neg if style_src else d_pos

    highest_overlap = 0
    closest_sentence = ""

    min_attribute_markers=len(get_attribute_markers(sentence, style_src))
    num_markers = 0

    previous_sentences = []
    backoff_count = 0
    while(num_markers < min_attribute_markers and backoff_count < param_backoff_limit):
        for sentence_b in opposite_dataset:
            weighted_overlap = get_weighted_overlap(sentence, sentence_b)
            if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
                highest_overlap = weighted_overlap
                closest_sentence = sentence_b
        highest_overlap = 0
        backoff_count += 1
        previous_sentences.append(closest_sentence)
        num_markers = len(get_attribute_markers(closest_sentence, not style_src))

    return closest_sentence

NameError: name 'd_both' is not defined

In [16]:
# Retrieve using tfidf
def retrieve(sentence, style_src):
    return separate(get_closest_sentence_tfidf(sentence, style_src), not style_src)

# Input Dict: Creating the lang dictionary

In [15]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [16]:
# count the overlap; we need function for this;
def get_overlap(a, b):
    # print(a, b)
    a_counter = collections.Counter(a.split())
    # print(a_counter)
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap


def get_total_overlap(a, b):
    return len(list(get_overlap(a, b).elements()))

In [19]:
noise_chance = 0.1

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    d_pos_a = []

    pairs_pos = []
    for sentence in nonclickbait_l_c:
        c = get_c(sentence, 1)
        a = get_a(sentence, 1)
        d_pos_a.append(a)
        pairs_pos.append([c, a, sentence])

    d_neg_a = []

    pairs_neg = []
    for sentence in clickbait_l_c:
        c = get_c(sentence, 0)
        a = get_a(sentence, 0)
        d_neg_a.append(a)
        pairs_neg.append([c, a, sentence])

    #adding noise for pos
    for pair in pairs_pos:
        if random.random() < noise_chance:
            real_a = pair[1].split()

            if(len(real_a) == 0):
                continue

            for a in d_pos_a:
                if(len(a) == 0):
                    continue
                a = a.split()

                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)


    #adding noise for neg
    for pair in pairs_neg:
        if random.random() < noise_chance:
            real_a = pair[1].split()
            if(len(real_a) == 0):
                continue

            for a in d_neg_a:
                if(len(a) == 0):
                    continue
                a = a.split()

                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    #concat atas bawah;
    pairs = np.concatenate((pairs_pos, pairs_neg), 0)

    for pair in pairs:

        input_lang.addSentence(pair[0])

        input_lang.addSentence(pair[1])
        # sentence full
        output_lang.addSentence(pair[2])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

get the input_dict, output_dict, and pairs ()

In [20]:
input_lang, output_lang, pairs = prepareData()

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_97668/2150605022.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(counts)


Counted words:
input 13961
output 13896


In [21]:
for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
for pair in pairs:
    input_lang.addSentence(pair[1])
    output_lang.addSentence(pair[0])

In [22]:
print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 13961
output 13984


In [23]:
pairs

array([['masuk radar medan , menantu jokowi bertemu dpw nasdem sumut',
        'pilwalkot',
        'masuk radar pilwalkot medan , menantu jokowi bertemu dpw nasdem sumut'],
       ['malaysia sudutkan ri : isu hingga invasi babi', 'kabut asap',
        'malaysia sudutkan ri : isu kabut asap hingga invasi babi'],
       ['ke ensos , 3 bagi korban kerusuhan sosial di ',
        'm salurkan rp 7 papua',
        'kemensos salurkan rp 7 , 3 m bagi korban kerusuhan sosial di papua'],
       ...,
       [" rilis teaser ' ' jelang , konsep dan koreografinya banjir pujian",
        'twice mv feel special comeback',
        "twice rilis teaser mv ' feel special ' jelang comeback , konsep dan koreografinya banjir pujian"],
       ['ada niat tambah momongan tanpa ikut program hamil , fedi nuril : kalau pengin ya buat ',
        'aja',
        'ada niat tambah momongan tanpa ikut program hamil , fedi nuril : kalau pengin ya buat aja'],
       [' akan segera , soobin akui gatal ingin spoiler',
     

In [24]:
from torch.utils.data import Subset

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [25]:
 from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)

In [26]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 6029
Number of testing examples: 2584


# Neural Network: Encoder Decoder

In [28]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [29]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.LSTM = nn.LSTM(word_vec_size,hidden_size, num_layers=2)

        self.embedding = nn.Embedding(input_size, word_vec_size)
        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [30]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, word_vec_size)

        # self.LSTM = nn.LSTM(word_vec_size,hidden_size, num_layers=2)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [31]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_c_tensor = tensorFromSentence(input_lang, pair[0])
    input_a_tensor = tensorFromSentence(input_lang, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    return input_c_tensor, input_a_tensor, target_tensor

In [32]:
#Training
MAX_LENGTH = 50
teacher_forcing_ratio = 0.5

def train(input_c_tensor  ,
          input_a_tensor,
          target_tensor,
          encoder_c,
          encoder_a,
          decoder,
          encoder_c_optimizer,
          encoder_a_optimizer,
          decoder_optimizer,
          criterion,
          max_length=MAX_LENGTH
          ):

    # init hidden
    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    # zero grad
    encoder_a_optimizer.zero_grad()
    encoder_c_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # len of sentence of c, a, t, that already; e.g input_c_length= len(input_c_tensor)
    input_c_length = input_c_tensor.size(0)
    input_a_length = input_a_tensor.size(0)
    target_length = target_tensor.size(0)


    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    loss = 0

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei], encoder_c_hidden)
        encoder_c_outputs[ei] = encoder_c_output[0, 0]

    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(
            input_a_tensor[ei], encoder_a_hidden)
        encoder_a_outputs[ei] = encoder_a_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Teacher forcing: Feed the target as the next input
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    # Without teacher forcing: use its own predictions as the next input
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_c_optimizer.step()
    encoder_a_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [33]:
import time
import math
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Training : Training and TrainingIters

In [36]:
def trainIters(
        encoder_a : EncoderRNN,
        encoder_c : EncoderRNN,
        decoder : DecoderRNN,
        n_iters : int ,
        print_every : int =1000,
        plot_every : int =100,
        learning_rate : float =0.01,):

    start: float = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_c_optimizer : optim.Adadelta = optim.Adadelta(encoder_c.parameters(), lr= learning_rate)
    encoder_a_optimizer : optim.Adadelta = optim.Adadelta(encoder_a.parameters(), lr= learning_rate)
    decoder_optimizer :  optim.Adadelta = optim.Adadelta(decoder.parameters(), lr= learning_rate)

    criterion: nn.NLLLoss = nn.NLLLoss()

    # taking e.g 100 of random pairs stc -> into tensors
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    for iter  in  range(1, n_iters + 1):
        # taking the pair and extract the c, a, target
        training_pair = training_pairs[iter - 1]
        input_tensor_c = training_pair[0]
        input_tensor_a = training_pair[1]
        target_tensor = training_pair[2]

        # train each of the tensors
        loss = train(

            input_tensor_c,
            input_tensor_a,
            target_tensor,

            encoder_a,
            encoder_c ,
            decoder,

            encoder_a_optimizer,
            encoder_c_optimizer,
            decoder_optimizer,

            criterion,
        )

        # get total loss and plot
        print_loss_total += loss
        plot_loss_total += loss

        # START : print and plotting every certain index
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

        showPlot(plot_losses)
        # END : plotting

Variable initialization for training Iterations

In [90]:
#INITIATION OF THE ENCODER DECODER
# prev size
word_vec_size = 128
hidden_size = 512
# word_size map of the input and the output of the words
INPUT_DIM = input_lang.n_words
OUTPUT_DIM = output_lang.n_words
# dimension_size of embedding layers
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128

# hidden_size of the dimension that connect between input, hidden,output
HID_DIM = 512
NN_LAYERS = 2

# implementation in encoder both A and C
encoder_c = EncoderRNN(INPUT_DIM, ENC_EMB_DIM, HID_DIM).to(device)
encoder_a = EncoderRNN(INPUT_DIM, DEC_EMB_DIM, HID_DIM).to(device)

# decoder
decoder = DecoderRNN(HID_DIM + HID_DIM, word_vec_size, OUTPUT_DIM).to(device)

In [38]:
# Running the Training -> trainingIters() -> train() : return -> None, but new develop encoder_a, encoder_c and decoder
n_iters = 10
print_every = 10
plot_every = 10
learning_rate = 0.01

trainIters(encoder_c, encoder_a, decoder, n_iters, print_every, plot_every, learning_rate)

0m 8s (- 1m 12s) (10 10%) 9.5520


/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_97668/2339069492.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_97668/2339069492.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


KeyboardInterrupt: 

In [94]:
def evaluateNoBeam(
        encoder_c : EncoderRNN,
        encoder_a : EncoderRNN,
        decoder : DecoderRNN,
        a: string,
        c : string,
        max_length=MAX_LENGTH):
    with torch.no_grad():

        # START: encoder from a,c
        input_tensor_c = tensorFromSentence(input_lang, c)
        input_tensor_a = tensorFromSentence(input_lang, a)

        input_length_c = input_tensor_c.size()[0]
        input_length_a = input_tensor_a.size()[0]

        encoder_hidden_c = encoder_c.initHidden()
        encoder_hidden_a = encoder_a.initHidden()

        encoder_outputs_c = torch.zeros(max_length, encoder_hidden_a.hidden_size, device=device)
        encoder_outputs_a = torch.zeros(max_length, encoder_hidden_c.hidden_size, device=device)

        for ei in range(input_length_c):
            encoder_output_c, encoder_hidden_c = encoder_c(input_tensor_c[ei], encoder_hidden_c )
            encoder_outputs_c[ei] += encoder_output_c[0, 0]

        for ei in range(input_length_a):
            encoder_output_a, encoder_hidden_a = encoder_a(input_tensor_a[ei], encoder_hidden_a)
            encoder_outputs_a[ei] += encoder_output_a[0, 0]

        enc_step = torch.cat((encoder_hidden_c, encoder_hidden_a), 2)
        # END : get encoder from a, c

        decoder_hidden = encoder_hidden

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoded_words = []

        # No Beam Search
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden)

            topv, topi =  F.softmax(decoder_output).data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

#  ======= DRAFT =======

In [ ]:
# testing example
stc_rand: [] = random.choice(pairs)
print("content : {} \nattribute: {} \ntgt: {} ".format(stc_rand[0], stc_rand[1], stc_rand[2]))

training_pairs = [tensorsFromPair(stc_rand) for i in range(10)]
# print(len(training_pairs))

iter = 8
training_pair = training_pairs[iter - 1]
input_tensor_c = training_pair[0]
# input_tensor_a = training_pair[1]
# target_tensor = training_pair[2]


# print(len(input_tensor_c))
# print(input_tensor_c)

# input_c_length = input_tensor_c.size(0)

# print(input_c_length)

# prev size
word_vec_size = 128
hidden_size = 512

# word_size map of the input and the output of the words
INPUT_DIM = input_lang.n_words
OUTPUT_DIM = output_lang.n_words

# dimension_size of embedding layers
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128

# hidden_size of the dimension that connect between input, hidden,output
HID_DIM = 512
NN_LAYERS = 2


# implementation in encoder both A and C
encoder_c = EncoderRNN(INPUT_DIM, ENC_EMB_DIM, HID_DIM).to(device)
encoder_a = EncoderRNN(INPUT_DIM, DEC_EMB_DIM, HID_DIM).to(device)

# decoder
decoder = DecoderRNN(HID_DIM + HID_DIM, word_vec_size, OUTPUT_DIM).to(device)

max_length = 50

encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)

